# Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
from read_data import read_file
from english_text_normalization import text_normalization
from text_processing import TextProcessing
import random
from tensorflow.keras.layers import LSTM, GRU, Attention, AdditiveAttention, MultiHeadAttention # type: ignore
from encoder_decoder_model import EncoderDecoderWithAttention, EncoderDecoderWithoutAttention
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
import warnings
import os
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Ignore Warnings

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

# Set Early Stop For Encoder-Decoder Model

In [3]:
early_stops = EarlyStopping(monitor='loss', mode = 'min', patience = 5)

# Read Data

In [4]:
Corpus = read_file(file_path='Question_Answering_Pairs.txt', text_normalization=text_normalization)

# Split The Data And Add Start And End Token

In [5]:
encoder_input, encoder_output = map(list, zip(*[pair.split('<sep>') for pair in Corpus])) # Split Data Into Input And Output
encoder_output = ["<sos>" + " " + line + " " + "<eos>"  for line in encoder_output] # Add Start And End Token To The Output

# Process The Data 

**Input Data**

In [6]:
text_processing_input = TextProcessing(data=encoder_input) # Input TextProcessing
all_words_input , words_input = text_processing_input.data_words(filter='[!"#$&*+,/:;=@[\\]^_`{|}~]') # Set With Unique Words And Find Number Of Unique Words
words_to_index_input = text_processing_input.words_to_index_(words=words_input) # Conver Word To Integer Index
index_to_words_input = text_processing_input.index_to_word_(words=words_input)
input_sequences = text_processing_input.text_to_sequence(word_index=words_to_index_input) # Convert Text Into Squences Of Integer
encoder_max_length = len(max(encoder_input, key = len)) # Find Max Length
pad_encoder_input = text_processing_input.sequences_padding(input_sequence=input_sequences, max_length=encoder_max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)
word_count_input = text_processing_input.word_counts() # Number Of Occurrence Each Word

**Output Data**

In [7]:
text_processing_output = TextProcessing(data=encoder_output) # Input TextProcessing
all_words_output , words_output = text_processing_output.data_words(filter='[!"#$%&()*+,./:;=@[\\]^_`{|}~]') # Set With Unique Words And Find Number Of Unique Words
words_to_index_output = text_processing_output.words_to_index_(words=words_output) # Conver Word To Integer Index
output_sequences = text_processing_output.text_to_sequence(word_index=words_to_index_output) # Convert Text Into Squences Of Integer
word_count_output = text_processing_output.word_counts() # Number Of Occurrence Each Word

In [8]:
all_words_output

609

# Generate Decoder Input And Output

In [9]:
def generate_decoder_inputs_targets(sequence):  # Generate Decoder Input And Output From Input Sequence
  decoder_inputs = [sentences[:-1] for sentences in sequence] 
  decoder_targets = [sentences[1:] for sentences in sequence]
  return decoder_inputs, decoder_targets
decoder_inputs, decoder_output = generate_decoder_inputs_targets(sequence= output_sequences)
decoder_max_length = len(max(decoder_inputs, key = len)) # Find Max Length Of Decoder
decoder_max_length

13

# Padding Decoder Data

In [10]:
pad_decoder_input = text_processing_output.sequences_padding(input_sequence=decoder_inputs, max_length=decoder_max_length)
pad_decoder_output = text_processing_output.sequences_padding(input_sequence=decoder_output, max_length=decoder_max_length)

# Encoder-Decoder Without Attention

**LSTM**

In [11]:
model_encoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
model_decoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
encoder_decoder_lstm = EncoderDecoderWithoutAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm, model_decoder = model_decoder_lstm, unit = all_words_output)
encoder_inputs, encoder_states = encoder_decoder_lstm.encoder()
decoder_inputs, decoder_outputs = encoder_decoder_lstm.decoder(encoder_states=encoder_states)
model_lstm_1 = encoder_decoder_lstm.build_model()
encoder_decoder_lstm.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_1, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_lstm.model_fit(model = model_lstm_1, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 11s 124ms/step - loss: 3.9173 - sparse_categorical_accuracy: 0.3864
Epoch 2/100
54/54 [==============================] - 7s 127ms/step - loss: 1.8354 - sparse_categorical_accuracy: 0.7761
Epoch 3/100
54/54 [==============================] - 7s 135ms/step - loss: 1.3897 - sparse_categorical_accuracy: 0.7761
Epoch 4/100
54/54 [==============================] - 6s 118ms/step - loss: 1.1973 - sparse_categorical_accuracy: 0.7761
Epoch 5/100
54/54 [==============================] - 7s 122ms/step - loss: 1.0935 - sparse_categorical_accuracy: 0.7814
Epoch 6/100
54/54 [==============================] - 6s 117ms/step - loss: 1.0103 - sparse_categorical_accuracy: 0.7981
Epoch 7/100
54/54 [==============================] - 6s 115ms/step - loss: 0.9352 - sparse_categorical_accuracy: 0.8305
Epoch 8/100
54/54 [==============================] - 6s 112ms/step - loss: 0.8847 - sparse_categorical_accuracy: 0.8359
Epoch 9/100
54/54 [====================

**GRU**

In [12]:
model_encoder_gru = GRU(units=128, return_sequences=True, return_state=True)
model_decoder_gru = GRU(units=128, return_sequences=True, return_state=True)
encoder_decoder = EncoderDecoderWithoutAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm, model_decoder = model_decoder_lstm, unit = all_words_output)
encoder_inputs, encoder_states = encoder_decoder.encoder()
decoder_inputs, decoder_outputs = encoder_decoder.decoder(encoder_states=encoder_states)
model_gru_1 = encoder_decoder.build_model()
encoder_decoder.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_1, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder.model_fit(model = model_gru_1, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 11s 131ms/step - loss: 3.4221 - sparse_categorical_accuracy: 0.5046
Epoch 2/100
54/54 [==============================] - 6s 119ms/step - loss: 1.4819 - sparse_categorical_accuracy: 0.7761
Epoch 3/100
54/54 [==============================] - 6s 119ms/step - loss: 1.2155 - sparse_categorical_accuracy: 0.7752
Epoch 4/100
54/54 [==============================] - 6s 116ms/step - loss: 1.0798 - sparse_categorical_accuracy: 0.7905
Epoch 5/100
54/54 [==============================] - 6s 114ms/step - loss: 0.9687 - sparse_categorical_accuracy: 0.8301
Epoch 6/100
54/54 [==============================] - 6s 113ms/step - loss: 0.8974 - sparse_categorical_accuracy: 0.8338
Epoch 7/100
54/54 [==============================] - 6s 113ms/step - loss: 0.8540 - sparse_categorical_accuracy: 0.8354
Epoch 8/100
54/54 [==============================] - 6s 113ms/step - loss: 0.8288 - sparse_categorical_accuracy: 0.8381
Epoch 9/100
54/54 [====================

# Encoder-Decoder With Attention

**1-Luong Attension**

*LSTM*

In [13]:
model_encoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
model_decoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
attention_layer = Attention()
encoder_decoder = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm, model_decoder = model_decoder_lstm, unit = all_words_output, attention_layer = attention_layer)
encoder_output, encoder_inputs, encoder_states = encoder_decoder.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder.decoder_(encoder_states=encoder_states, encoder_outputs=encoder_output)
model_lstm_2 = encoder_decoder.build_model()
encoder_decoder.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_2, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder.model_fit(model = model_lstm_2, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 11s 118ms/step - loss: 3.5835 - sparse_categorical_accuracy: 0.4404
Epoch 2/100
54/54 [==============================] - 6s 118ms/step - loss: 1.5784 - sparse_categorical_accuracy: 0.7761
Epoch 3/100
54/54 [==============================] - 6s 118ms/step - loss: 1.2112 - sparse_categorical_accuracy: 0.7756
Epoch 4/100
54/54 [==============================] - 6s 116ms/step - loss: 1.0899 - sparse_categorical_accuracy: 0.7795
Epoch 5/100
54/54 [==============================] - 6s 114ms/step - loss: 1.0378 - sparse_categorical_accuracy: 0.7882
Epoch 6/100
54/54 [==============================] - 6s 114ms/step - loss: 1.0013 - sparse_categorical_accuracy: 0.7893
Epoch 7/100
54/54 [==============================] - 6s 112ms/step - loss: 0.9691 - sparse_categorical_accuracy: 0.8009
Epoch 8/100
54/54 [==============================] - 6s 112ms/step - loss: 0.9296 - sparse_categorical_accuracy: 0.8088
Epoch 9/100
54/54 [====================

*GRU*

In [14]:
model_encoder_gru = GRU(units=128, return_sequences=True, return_state=True)
model_decoder_gru = GRU(units=128, return_sequences=True, return_state=True)
attention_layer = Attention()
encoder_decoder = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_gru, model_decoder = model_decoder_gru, unit = all_words_output, attention_layer = attention_layer)
encoder_output, encoder_inputs, encoder_states = encoder_decoder.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder.decoder_(encoder_states=encoder_states, encoder_outputs=encoder_output)
model_gru_2 = encoder_decoder.build_model()
encoder_decoder.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_2, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder.model_fit(model = model_gru_2, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 9s 103ms/step - loss: 3.5710 - sparse_categorical_accuracy: 0.5361
Epoch 2/100
54/54 [==============================] - 5s 87ms/step - loss: 1.3234 - sparse_categorical_accuracy: 0.7761
Epoch 3/100
54/54 [==============================] - 5s 85ms/step - loss: 1.0998 - sparse_categorical_accuracy: 0.7769
Epoch 4/100
54/54 [==============================] - 5s 88ms/step - loss: 1.0257 - sparse_categorical_accuracy: 0.7840
Epoch 5/100
54/54 [==============================] - 5s 98ms/step - loss: 0.9603 - sparse_categorical_accuracy: 0.8072
Epoch 6/100
54/54 [==============================] - 6s 108ms/step - loss: 0.8737 - sparse_categorical_accuracy: 0.8350
Epoch 7/100
54/54 [==============================] - 6s 110ms/step - loss: 0.8043 - sparse_categorical_accuracy: 0.8361
Epoch 8/100
54/54 [==============================] - 6s 107ms/step - loss: 0.7603 - sparse_categorical_accuracy: 0.8388
Epoch 9/100
54/54 [=========================

**2-Bahdanau Attension**

*LSTM*

In [15]:
model_encoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
model_decoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
attention_layer = AdditiveAttention()
encoder_decoder = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm, model_decoder = model_decoder_lstm, unit = all_words_output, attention_layer = attention_layer)
encoder_output, encoder_inputs, encoder_states = encoder_decoder.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder.decoder_(encoder_states=encoder_states, encoder_outputs=encoder_output)
model_lstm_3 = encoder_decoder.build_model()
encoder_decoder.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_3, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder.model_fit(model = model_lstm_3, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 13s 155ms/step - loss: 3.7096 - sparse_categorical_accuracy: 0.4685
Epoch 2/100
54/54 [==============================] - 8s 144ms/step - loss: 1.6750 - sparse_categorical_accuracy: 0.7761
Epoch 3/100
54/54 [==============================] - 8s 143ms/step - loss: 1.2395 - sparse_categorical_accuracy: 0.7761
Epoch 4/100
54/54 [==============================] - 8s 141ms/step - loss: 1.0590 - sparse_categorical_accuracy: 0.7827
Epoch 5/100
54/54 [==============================] - 7s 138ms/step - loss: 0.9630 - sparse_categorical_accuracy: 0.7952
Epoch 6/100
54/54 [==============================] - 8s 140ms/step - loss: 0.8950 - sparse_categorical_accuracy: 0.8126
Epoch 7/100
54/54 [==============================] - 8s 141ms/step - loss: 0.8379 - sparse_categorical_accuracy: 0.8293
Epoch 8/100
54/54 [==============================] - 7s 136ms/step - loss: 0.7945 - sparse_categorical_accuracy: 0.8352
Epoch 9/100
54/54 [====================

*GRU*

In [16]:
model_encoder_gru = GRU(units=128, return_sequences=True, return_state=True)
model_decoder_gru = GRU(units=128, return_sequences=True, return_state=True)
attention_layer = Attention()
encoder_decoder = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_gru, model_decoder = model_decoder_gru, unit = all_words_output, attention_layer = attention_layer)
encoder_output, encoder_inputs, encoder_states = encoder_decoder.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder.decoder_(encoder_states=encoder_states, encoder_outputs=encoder_output)
model_gru_3 = encoder_decoder.build_model()
encoder_decoder.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_3, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder.model_fit(model = model_gru_3, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 10s 102ms/step - loss: 3.6373 - sparse_categorical_accuracy: 0.6023
Epoch 2/100
54/54 [==============================] - 5s 90ms/step - loss: 1.4028 - sparse_categorical_accuracy: 0.7761
Epoch 3/100
54/54 [==============================] - 6s 114ms/step - loss: 1.1055 - sparse_categorical_accuracy: 0.7783
Epoch 4/100
54/54 [==============================] - 6s 116ms/step - loss: 0.9817 - sparse_categorical_accuracy: 0.7927
Epoch 5/100
54/54 [==============================] - 6s 113ms/step - loss: 0.8766 - sparse_categorical_accuracy: 0.8261
Epoch 6/100
54/54 [==============================] - 6s 102ms/step - loss: 0.7990 - sparse_categorical_accuracy: 0.8399
Epoch 7/100
54/54 [==============================] - 6s 108ms/step - loss: 0.7547 - sparse_categorical_accuracy: 0.8423
Epoch 8/100
54/54 [==============================] - 6s 106ms/step - loss: 0.7284 - sparse_categorical_accuracy: 0.8444
Epoch 9/100
54/54 [=====================

**3-MultiHead Attesnion**

*LSTM*

In [17]:
model_encoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
model_decoder_lstm = LSTM(units=128, return_sequences=True, return_state=True)
attention_layer = MultiHeadAttention(num_heads=5, key_dim=64)
encoder_decoder = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm, model_decoder = model_decoder_lstm, unit = all_words_output, attention_layer = attention_layer)
encoder_output, encoder_inputs, encoder_states = encoder_decoder.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder.decoder_(encoder_states=encoder_states, encoder_outputs=encoder_output)
model_lstm_4 = encoder_decoder.build_model()
encoder_decoder.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_4, metrics=['sparse_categorical_accuracy'])
with tf.device('cpu'):
    history = encoder_decoder.model_fit(model = model_lstm_4, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 22s 344ms/step - loss: 3.3303 - sparse_categorical_accuracy: 0.4981
Epoch 2/100
54/54 [==============================] - 18s 335ms/step - loss: 1.3958 - sparse_categorical_accuracy: 0.7707
Epoch 3/100
54/54 [==============================] - 18s 329ms/step - loss: 1.1280 - sparse_categorical_accuracy: 0.7767
Epoch 4/100
54/54 [==============================] - 18s 329ms/step - loss: 0.9957 - sparse_categorical_accuracy: 0.7840
Epoch 5/100
54/54 [==============================] - 18s 334ms/step - loss: 0.9139 - sparse_categorical_accuracy: 0.8065
Epoch 6/100
54/54 [==============================] - 18s 341ms/step - loss: 0.8389 - sparse_categorical_accuracy: 0.8301
Epoch 7/100
54/54 [==============================] - 18s 339ms/step - loss: 0.7798 - sparse_categorical_accuracy: 0.8389
Epoch 8/100
54/54 [==============================] - 19s 343ms/step - loss: 0.7341 - sparse_categorical_accuracy: 0.8425
Epoch 9/100
54/54 [=============

*GRU*

In [18]:
model_encoder_gru = GRU(units=128, return_sequences=True, return_state=True)
model_decoder_gru = GRU(units=128, return_sequences=True, return_state=True)
attention_layer = MultiHeadAttention(num_heads=5, key_dim=64)
encoder_decoder = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = encoder_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_gru, model_decoder = model_decoder_gru, unit = all_words_output, attention_layer = attention_layer)
encoder_output, encoder_inputs, encoder_states = encoder_decoder.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder.decoder_(encoder_states=encoder_states, encoder_outputs=encoder_output)
model_gru_4 = encoder_decoder.build_model()
encoder_decoder.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_4, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder.model_fit(model = model_gru_4, epochs = 100, early_stop = early_stops, batch_size = 32, encoder_input = pad_encoder_input, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
54/54 [==============================] - 16s 190ms/step - loss: 3.4956 - sparse_categorical_accuracy: 0.5121
Epoch 2/100
54/54 [==============================] - 8s 155ms/step - loss: 1.2673 - sparse_categorical_accuracy: 0.7772
Epoch 3/100
54/54 [==============================] - 8s 153ms/step - loss: 1.0353 - sparse_categorical_accuracy: 0.7803
Epoch 4/100
54/54 [==============================] - 8s 152ms/step - loss: 0.9025 - sparse_categorical_accuracy: 0.8107
Epoch 5/100
54/54 [==============================] - 8s 155ms/step - loss: 0.8053 - sparse_categorical_accuracy: 0.8321
Epoch 6/100
54/54 [==============================] - 9s 171ms/step - loss: 0.7404 - sparse_categorical_accuracy: 0.8420
Epoch 7/100
54/54 [==============================] - 9s 169ms/step - loss: 0.6962 - sparse_categorical_accuracy: 0.8488
Epoch 8/100
54/54 [==============================] - 9s 167ms/step - loss: 0.6511 - sparse_categorical_accuracy: 0.8570
Epoch 9/100
54/54 [====================